COCONUT AGE CLASSFICATION

## IMPORTS

In [29]:
import sys

!{sys.executable} -m pip install opencv-python
!"{sys.executable}" -m pip install opencv-python
%pip install tensorflow






'c:\Users\Mary' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



^C
Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached wrapt-1.17.3-cp313-cp313-win_amd64.whl.metadata (6.5 kB)
  Using cac

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install tensorflow

import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'tensorflow'

## PREPROCESSING

In [ ]:
# direction in which the folder where images are stored
image_dir = 'images' #this  depend kung asa siya gi butang
IMG_SIZE = (224, 224)


# ----------------------------------
# IMAGE PREPROCESSING
#-----------------------------------

img_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),   # Flipping
    layers.RandomRotation(0.2),                     # Rotation
    layers.RandomZoom(0.2),                         # Zooming
    layers.RandomContrast(0.2),                     # Contrast
    layers.RandomTranslation(0.1, 0.1),             # Shifting
    layers.Resizing(224, 224),                      # Resize to MobileNet input size
    layers.Rescaling(1./255, offset=-1),            # Normalization to [-1,1]
    layers.CenterCrop(224, 224)
])
   


In [ ]:
pip install opencv-python

In [ ]:
# Load metadata
df = pd.read_csv('metadata.csv') #Change to the .csv file


# ----------------------------------
# NUMERICAL PREPROCESSING
#-----------------------------------

# Step 1: Applying the PCA Formula: Age = TrunkHeight / L11 + 5
df["age_formula"] = df["HeightTrunk"] / df["11leafscars"] + 5

# Step 2: Apply the General regression
aprox_age = "Year" in df.columns and df["Year"].notna().any()

if approx_age:
  df["actual_age"] = 2025 - df["Year"]
  mask = df["actual_age"].notna() & np.isinfinite(df["actual_age"])
  x = df.loc[mask,["age_formula"]]
  y = df.loc[mask,["actual_age"]]
  reg = LinearRegression().fit(x,y)
  a = float(ref.coef_[0]); b = float(reg,intercept)

else:
  # No labels at all, it will fall back to identity (no correction needed)
  a,b = 1.0, 0.0

# Step 3: Apply the linear regression formula a * (age_formula) + b (without
# planting year)

# Step 4: Show the results
print("Calibrated a, b:", a, b)
print(df[["age_formula", "actual_age"]])